# for Greenwood and Jovanovich


Other refereces:
- Integral over a distribution: http://math.stackexchange.com/questions/380785/what-does-it-mean-to-integrate-with-respect-to-the-distribution-function


In [ ]:
import numpy as np
import scipy as sp
import scipy.stats
from scipy.optimize import fminbound

In [ ]:
# Interpolation

grid = np.linspace(0,10,11)
w = 5 * grid - 25
print sp.interp(2, grid, w)
print type(grid)
print grid.size


In [ ]:
# Normal distributions density funciton.
import scipy.stats

def exp_val(x,safe):
    print x,":",max(safe,x),"*",sp.stats.norm(loc=2,scale=0.3).pdf(x),"=",((max(safe,x))*sp.stats.norm(loc=2,scale=0.3).pdf(x))
    return ((max(safe,x))*sp.stats.norm(loc=2,scale=0.3).pdf(x))
    
def int_exp_val(safe):
    return sp.integrate.quad(exp_val,0,5,args=(safe))[0]
    
grid = np.random.normal(2,0.3,size=60)

wealth_distr = scipy.stats.norm(loc=2,scale=0.4)

#print min(grid)
print int_exp_val(0)
print sp.integrate.quad(lambda x: sp.stats.norm(loc=2,scale=0.3).pdf(x),0,5)[0]


In [ ]:
print sp.stats.norm(loc=2,scale=0.3).pdf(4.191)

Tests with the interpolation function and the integral function.

In [ ]:
    
x_axis = np.linspace(0,15,500)
y_axis = x_axis*0.5
print y_axis

In [ ]:
def x_interp(xax,yax):
    return lambda x: sp.interp(x,xax,yax)

function_ = x_interp(x_axis,y_axis)
print function_(15)


In [ ]:
SAFE = 1
def PDF(x):
    return sp.stats.norm(loc=2,scale=0.3).pdf(x)
def integral(s,am):
    return sp.integrate.quad((lambda theta,sI : sp.interp(sI*max(SAFE,theta),x_axis,y_axis)*PDF(theta)),0,am,args=s)

print integral(1,10)

In [ ]:
BETA = 0.987
capital = 3
objective = lambda s,k: - np.log(k-s) - BETA * integral(s,10)[0]
s_star = fminbound(objective, 1e-6, capital, args=(capital,))
print s_star
print -objective(s_star,capital)

## The Bellman operators for the two value functions
This part is based on the Prof. Sargent and Prof. Stachurski's approach from [HERE](http://quant-econ.net/py/optgrowth.html).

THIS IS A FINAL VERSION FOR THE the V(K) FUNCTION!

In [4]:
import numpy as np
import scipy as sp
import scipy.stats
from scipy.integrate import quad
from scipy.optimize import fminbound
from numpy import log
from scipy import interp

SAFE = 1
BETA = 0.987
# DISTRIBUTION PARAM
MEAN = 2
STDEV = 0.3
MAX_VAL = 5
# Number of iterations
N = 200

wealth_axis = np.linspace(1e-6,15,10)

def PDF(x):
    return scipy.stats.norm(loc=MEAN,scale=STDEV).pdf(x)

def v_integral(s,MAX,W_ax,Y_ax):
    return quad((lambda theta,sI : interp(sI*max(SAFE,theta),
                                        W_ax,Y_ax)*PDF(theta)),0,MAX,args=(s,),limit=100)

def v_bellman_op(v):
    Tv = np.empty(wealth_axis.size)
    for i, k in enumerate(wealth_axis):
        objective = lambda s,k: - np.log(k-s) - BETA * v_integral(s,MAX_VAL,
                                                               wealth_axis,
                                                               v)[0]
        s_star = fminbound(objective, 1e-12, k-1e-12, args=(k,))
        Tv[i] = -objective(s_star,k)
        print i
    return Tv

v = 5 * log(wealth_axis) - 25
for i in range(N):
    v = v_bellman_op(v)
np.savetxt('v_value_func.csv',v, delimiter=",")
np.savetxt('wealth_grid.csv',wealth_axis, delimiter=",")

0
1
2
3
4
5
6
7
8
9
[-94.07755279 -22.44586921 -18.98013481 -16.95280977 -15.51439966
 -14.39868205 -13.48707437 -12.71632104 -12.04866413 -11.45974899]
[-107.15126733  -21.1808523   -16.89829775  -14.46906857  -12.75540259
  -11.51489556  -10.72121943  -10.20276934   -9.83360382   -9.55174137]


The multiprocessing version can be found in the python file.

[-94.07755279 -30.91883449 -27.45310675 -25.42578393 -23.98737493
 -22.87165799 -21.96005075 -21.18929774 -20.52164107 -19.93272612
 -19.40592372 -18.92937297 -18.49431621 -18.09410277 -17.723563
 -17.37859872 -17.05590619 -16.75278314 -16.46699112 -16.19665506
 -15.94018863 -15.69623785 -15.46363781 -15.24137903 -15.02858099
 -14.82447104 -14.6283675  -14.43966588 -14.25782768 -14.08237111
 -13.91286337 -13.74891427 -13.59017079 -13.43631252 -13.28704771
 -13.14211004 -13.00125567 -12.86426081 -12.73091959 -12.60104217
 -12.47445314 -12.35099009 -12.23050234 -12.11284986 -11.99790228
 -11.88553801 -11.77564348 -11.66811246 -11.56284542 -11.45974899]
[-69.07755279541016, -5.918834686279297, -2.453106641769409, -0.4257839322090149, 1.0126250982284546, 2.1283419132232666, 3.0399491786956787, 3.810702323913574, 4.478358745574951, 5.06727409362793, 5.594076156616211, 6.070627212524414, 6.505683898925781, 6.90589714050293, 7.276436805725098, 7.621401309967041, 7.944093704223633, 8.247217178